In [1]:
"""
Jason Stranne
"""
import numpy as np
import os
import sys
import gc
from RP_Downstream_Trainer import DownstreamNet, Downstream_Dataset, print_class_counts, num_correct, reduce_dataset_size
from RP_Downstream_Trainer import smallest_class_len, restrict_training_size_per_class, train_end_to_end
sys.path.insert(0, '..')
from Stager_net_pratice import StagerNet
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import LeaveOneGroupOut, GroupKFold
from torch import Tensor
from torch.utils.data import DataLoader, TensorDataset


In [2]:
def train_different_classes(model_path, tset, vset, epochs, sample_list):
    outList=[]
    balanced_acc_out=[]
    for i in sample_list:
        # print(i)
        acc, balanced_acc = train_end_to_end(model_path, tset, vset, i, epochs, 3)
        outList.append(acc)
        balanced_acc_out.append(balanced_acc)
    return outList, balanced_acc_out

In [3]:
root = os.path.join("Mouse_Training_Data", "Windowed_Data", "")
datasets_list=[]
print('Loading Data')
f=open(os.path.join("training_names.txt"),'r')
lines = f.readlines()
x_vals = []
y_vals = []
groups = []
index = 0
for line in lines:
    recordName=line.strip()
    print('Processing', recordName)
    data_file=root+recordName+os.sep+recordName
    d = Downstream_Dataset(path=data_file)
    x_vals.append(d.data)
    y_vals.append(d.labels)
    groups.append(np.ones(len(d.labels))*index)
    index+=1
    
f.close()


x_vals = np.vstack(x_vals)
y_vals = np.concatenate(y_vals, axis=0)
groups = np.concatenate(groups, axis=0)
print(x_vals.shape)
print(y_vals.shape)
print(groups.shape)


# logo = LeaveOneGroupOut()
kfold = GroupKFold(n_splits=5)
# logo.get_n_splits(x_vals, y_vals, groups)
kfold.get_n_splits(x_vals, y_vals, groups)

result_dict = {}
# dtype=torch.int32
for train_index, test_index in kfold.split(x_vals, y_vals, groups):
    unique = np.unique(groups[test_index])
    print("Leaving out mouse number:", unique)
    training_set = TensorDataset(torch.tensor(x_vals[train_index], dtype=torch.float), torch.tensor(y_vals[train_index], dtype=torch.long))
    test_set = TensorDataset(torch.tensor(x_vals[test_index], dtype=torch.float), torch.tensor(y_vals[test_index], dtype=torch.long))
    print(len(training_set))
    smallest_class = smallest_class_len(training_set, 3)
    num_samples=[]
    temp=1
    while temp < smallest_class:
        num_samples.append(temp)
        temp*=10
    num_samples.append(None)
    accuracy, balanced_accuracy = train_different_classes("full_supervision", training_set, test_set, 100, num_samples)
    for num_pos, acc in zip(num_samples, accuracy):
        if num_pos not in result_dict:
            result_dict[num_pos] = []
        result_dict[num_pos].append(acc)
    print(result_dict)
    
for k in result_dict:
    result_dict[k] = np.mean(result_dict[k])
print(result_dict)


Loading Data
Processing MouseCKA1_030515_HCOFTS
removed 0 unknown entries
Processing MouseCKB9_022715_HCOFTS
removed 0 unknown entries
Processing MouseCKL1_062514_HCOFTS
removed 0 unknown entries
Processing MouseCKL5_063014_HCOFTS
removed 0 unknown entries
Processing MouseCKL7_063014_HCOFTS
removed 0 unknown entries
Processing MouseCKN1_063014_HCOFTS
removed 0 unknown entries
Processing MouseCKN2_070214_HCOFTS
removed 0 unknown entries
Processing MouseCKN3_070214_HCOFTS
removed 0 unknown entries
Processing MouseCKO1_070214_HCOFTS
removed 0 unknown entries
Processing MouseCKO2_070314_HCOFTS
removed 0 unknown entries
Processing MouseCKO3_070314_HCOFTS
removed 0 unknown entries
Processing MouseCKR1_082514_HCOFTS
removed 0 unknown entries
Processing MouseCKR3_082514_HCOFTS
removed 0 unknown entries
Processing MouseCKU10_092514_HCOFTS
removed 0 unknown entries
Processing MouseCKU1_092414_HCOFTS
removed 0 unknown entries
Processing MouseCKV10_101414_HCOFTS
removed 0 unknown entries
Processin

In [4]:
gc.collect()
{1: 0.39258333333333334, 10: 0.36705, 100: 0.2986666666666667, 1000: 0.8434833333333334, None: 0.8944333333333333}
{1: 0.2754666666666667, 10: 0.2736166666666667, 100: 0.40263333333333334, 1000: 0.8560833333333333, None: 0.8961833333333333}
{1: 0.28291666666666665, 10: 0.31415000000000004, 100: 0.3532, 1000: 0.8490500000000001, None: 0.8926166666666667}


60

In [4]:
trial1= [0.39258333333333334, 0.36705, 0.36408333333333337, 0.8434833333333334, 0.8944333333333333]
trial2=[0.33214999999999995, 0.3703333333333333, 0.40263333333333334, 0.8560833333333333, 0.8961833333333333]
trial3= [0.28291666666666665, 0.31415000000000004, 0.3532, 0.8490500000000001, 0.8926166666666667]
avg=[]
for i in range(len(trial1)):
    avg.append((trial1[i]+trial2[i]+trial3[i])/3)
print(avg)

[0.3358833333333333, 0.35051111111111116, 0.37330555555555556, 0.849538888888889, 0.8944111111111112]


In [5]:
RP_vals = train_different_classes("RP_stagernet.pth", training_set, validation_set, 100, num_samples)

1
len of the dataloader is: 1
Start Training
[0.5416666666666666]
10
len of the dataloader is: 1
Start Training
[0.5416666666666666, 0.8263888888888888]
None
len of the dataloader is: 1
Start Training
[0.5416666666666666, 0.8263888888888888, 0.8611111111111112]


{1: 0.41169696969696973, 10: 0.4972234848484849, 100: 0.6805530303030303, 1000: 0.854409090909091, None: 0.9312083333333334}

[0.41169696969696973, 0.4972234848484849, 0.6805530303030303, 0.854409090909091, 0.9312083333333334]